In [39]:
import sklearn
import pandas as pd
import sqlite3
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import svm
from datetime import datetime
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
import pickle
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder


import joblib

import warnings
warnings.filterwarnings('ignore')

In [121]:
df_train = pd.read_pickle("./train_data.pkl")
df_test = pd.read_pickle("./test_data.pkl")

In [122]:
df_raw = pd.read_pickle("./data.pkl")
df_raw

,titel,bouwjaar,kilometer_stand,vermogen,prijs,is_handgeschakeld,is_benzine,upload_datum,apk
1,fiat grande punto emotion automaat airco,2008,169504.0,78,2950.0,0.0,1,16,128
2,fiat grande punto edizione cool airco cruise,2007,219718.0,65,1950.0,1.0,1,11,183
3,fiat grande punto emotion airco,2006,199908.0,95,1895.0,1.0,1,11,219
4,fiat grande punto active airco rijdt prima,2008,138191.0,65,2740.0,1.0,1,11,381
5,fiat grande punto sportsound airco,2006,139057.0,65,2950.0,1.0,1,11,281
...,...,...,...,...,...,...,...,...,...
934,fiat punto street airco eigenaar,2016,109610.0,82,5975.0,1.0,0,16,147
935,fiat punto grande punto dynamic,2008,191463.0,78,2750.0,1.0,1,16,147
936,fiat punto,2011,210060.0,84,2700.0,1.0,0,16,147
937,fiat punto mylife,2011,167537.0,84,3495.0,1.0,0,16,147


In [148]:
target_names = ["titel", "bouwjaar", "kilometer_stand", "vermogen", "is_handgeschakeld", "is_benzine", "upload_datum", "apk"]

numeric_features = ['bouwjaar', "kilometer_stand", "vermogen", "upload_datum", "apk"]
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

preprocessor = ColumnTransformer(
    transformers=[
        ('numeric', numeric_transformer, numeric_features),
        ('text',  CountVectorizer(), 'titel')],
    remainder='passthrough')

In [157]:
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

X_train = df_train.drop(['prijs'], axis=1)
y_train = df_train[['prijs']]

X_test = df_test.drop(['prijs'], axis=1)
y_test = df_test[['prijs']]['prijs']
X_test

,titel,bouwjaar,kilometer_stand,vermogen,is_handgeschakeld,is_benzine,upload_datum,apk
357,fiat punto grande active deurs rijklaarprijs k...,2009,141940.0,67,1.0,1,16,147
851,fiat grande punto dynamic,2007,179086.0,65,1.0,1,10,163
780,fiat grande punto dynamic goed onderhouden,2006,160886.0,65,1.0,1,2,147
268,fiat grande punto actual airco deurs,2010,185314.0,84,1.0,0,25,600
41,fiat grande punto active,2007,232477.0,75,1.0,0,13,297
...,...,...,...,...,...,...,...,...
817,fiat grande punto dynamic airco clima cruise c...,2008,65513.0,77,1.0,1,5,57
26,fiat grande punto edizione cool deurs garantie,2006,117271.0,67,1.0,1,12,147
92,fiat grande punto dynamic airco cruise auto,2008,172488.0,77,1.0,1,18,169
11,fiat grande punto active,2007,238443.0,75,1.0,0,11,147


In [150]:
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

In [151]:
def evaluate_model(y_test, y_pred):
    print("Mean absolute error:\t", int(mean_absolute_error(y_test, y_pred)))
    print("Mean squared error:\t", int(mean_squared_error(y_test, y_pred)))

In [152]:
dtree = tree.DecisionTreeRegressor()
dtree.fit(X_train, y_train)
y_pred = dtree.predict(X_test)
evaluate_model(y_test, y_pred)

Mean absolute error:	 714
Mean squared error:	 1167120


In [153]:
random_forest = RandomForestRegressor(n_estimators=100)
random_forest.fit(X_train, y_train)
y_pred = random_forest.predict(X_test)
evaluate_model(y_test, y_pred)

Mean absolute error:	 542
Mean squared error:	 696271


In [154]:
joblib.dump(random_forest, 'RandomForestRegressor')
joblib.dump(scaler, 'RFScaler')
joblib.dump(preprocessor, 'RFPreprocessor')

['RFPreprocessor']

In [156]:
X_test.toarray()

array([[-0.27588236, -0.23731407, -1.2630681 , ...,  0.        ,
         1.        ,  1.        ],
       [-0.98053394,  0.09447198, -1.41816091, ...,  0.        ,
         1.        ,  1.        ],
       [-1.33285973, -0.06808943, -1.41816091, ...,  0.        ,
         1.        ,  1.        ],
       ...,
       [-0.62820815,  0.03553901, -0.48760404, ...,  0.        ,
         1.        ,  1.        ],
       [-0.98053394,  0.62464548, -0.64269685, ...,  0.        ,
         1.        ,  0.        ],
       [ 1.48574658, -0.42027604,  1.37350971, ...,  0.        ,
         1.        ,  1.        ]])